In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

In [2]:
from google.colab import files
uploaded = files.upload()

Saving track2_test_participant.csv to track2_test_participant.csv
Saving track2_train_participant.csv to track2_train_participant.csv


In [52]:
train = pd.read_csv('track2_train_participant.csv')
test = pd.read_csv('track2_test_participant.csv')

In [53]:
# 1) 문자열로 된 "0.6  < = t < = 0.7" 같은 구간에서 숫자만 뽑아주는 함수
def parse_range(s):
    nums = re.findall(r"[\d\.]+", s)
    return float(nums[0]), float(nums[1])

# 2) 두 그룹(GI 외, GI 오일링)에 대한 암페어 매트릭스를 숫자구간으로 정의
amp_matrix = {
    'GI 외': {
        (0.4, 0.5):    {(800,1850):200},
        (0.5, 0.6):    {(800,1600):200, (1600,1700):210, (1700,1800):220, (1800,1850):230},
        (0.6, 0.7):    {(800,1400):200, (1400,1500):220, (1500,1600):230, (1600,1700):250, (1700,1800):260, (1800,1850):270},
        (0.7, 0.8):    {(800,1200):200, (1200,1300):220, (1300,1400):230, (1400,1500):250, (1500,1600):260, (1600,1700):290, (1700,1800):300, (1800,1850):310},
        (0.8, 0.9):    {(800,1000):200, (1000,1100):210, (1100,1200):230, (1200,1300):250, (1300,1400):270, (1400,1500):280, (1500,1600):290, (1600,1700):310, (1700,1800):330, (1800,1850):340},
        (0.9, 1.0):    {(800,900):200, (900,1000):210, (1000,1100):230, (1100,1200):250, (1200,1300):270, (1300,1400):290, (1400,1500):310, (1500,1600):320, (1600,1700):340, (1700,1800):360, (1800,1850):370},
        (1.0, 1.1):    {(800,900):210, (900,1000):230, (1000,1100):250, (1100,1200):270, (1200,1300):290, (1300,1400):320, (1400,1500):340, (1500,1600):350, (1600,1700):360, (1700,1800):390, (1800,1850):400},
        (1.1, 1.2):    {(800,900):230, (900,1000):240, (1000,1100):280, (1100,1200):290, (1200,1300):310, (1300,1400):330, (1400,1500):350, (1500,1600):360, (1600,1700):390, (1700,1800):410, (1800,1850):420},
        (1.2, 1.3):    {(800,900):250, (900,1000):250, (1000,1100):300, (1100,1200):310, (1200,1300):320, (1300,1400):340, (1400,1500):370, (1500,1600):380, (1600,1700):400, (1700,1800):420, (1800,1850):430},
        (1.3, 1.4):    {(800,900):270, (900,1000):270, (1000,1100):310, (1100,1200):320, (1200,1300):330, (1300,1400):350, (1400,1500):380, (1500,1600):390, (1600,1700):410, (1700,1800):430, (1800,1850):430},
        (1.4, 1.5):    {(800,900):280, (900,1000):290, (1000,1100):320, (1100,1200):340, (1200,1300):350, (1300,1400):360, (1400,1500):390, (1500,1600):420, (1600,1700):430, (1700,1800):430, (1800,1850):430},
        (1.5, 1.6):    {(800,900):300, (900,1000):310, (1000,1100):330, (1100,1200):350, (1200,1300):360, (1300,1400):390, (1400,1500):410, (1500,1600):430, (1600,1700):430, (1700,1800):430, (1800,1850):430},
        (1.6, 1.7):    {(800,900):310, (900,1000):320, (1000,1100):340, (1100,1200):360, (1200,1300):380, (1300,1400):410, (1400,1500):430, (1500,1600):430, (1600,1700):430, (1700,1800):430, (1800,1850):430},
        (1.7, 1.8):    {(800,900):330, (900,1000):340, (1000,1100):350, (1100,1200):370, (1200,1300):400, (1300,1850):430},
        (1.8, 1.9):    {(800,900):340, (900,1100):360, (1100,1200):390, (1200,1850):430},
        (1.9, 2.0):    {(800,900):350, (900,1100):380, (1100,1200):410, (1200,1850):430}
    },
    'GI 오일링': {
        (0.6, 0.7):    {(800,1100):200, (1100,1200):210, (1200,1300):220, (1300,1400):240, (1400,1500):260, (1500,1600):280, (1600,1700):300, (1700,1800):320},
        (0.7, 0.8):    {(800,900):200, (900,1000):200, (1000,1100):220, (1100,1200):240, (1200,1300):260, (1300,1400):280, (1400,1500):300, (1500,1600):320, (1600,1700):330, (1700,1800):340},
        (0.8, 0.9):    {(800,900):200, (900,1000):220, (1000,1100):250, (1100,1200):270, (1200,1300):290, (1300,1400):320, (1400,1500):340, (1500,1600):360, (1600,1700):370, (1700,1800):380},
        (0.9, 1.0):    {(800,900):220, (900,1000):250, (1000,1100):280, (1100,1200):300, (1200,1300):330, (1300,1500):360, (1500,1600):370, (1600,1700):380, (1700,1800):390},
        (1.0, 1.1):    {(800,900):250, (900,1000):280, (1000,1100):300, (1100,1200):330, (1200,1300):360, (1300,1500):370, (1500,1600):380, (1600,1700):400, (1700,1800):430},
        (1.1, 1.2):    {(800,900):270, (900,1000):300, (1000,1100):330, (1100,1200):370, (1200,1400):370, (1400,1500):390, (1500,1600):410, (1600,1700):440},
        (1.2, 1.3):    {(800,900):290, (900,1000):330, (1000,1100):340, (1100,1200):370, (1200,1300):380, (1300,1400):390, (1400,1500):420, (1500,1700):450},
        (1.3, 1.4):    {(800,900):320, (900,1000):350, (1000,1100):360, (1100,1200):390, (1200,1300):390, (1300,1400):420, (1400,1700):450},
        (1.4, 1.5):    {(800,900):340, (900,1000):360, (1000,1100):370, (1100,1200):400, (1200,1300):420, (1300,1700):450},
        (1.5, 1.6):    {(800,900):370, (900,1000):370, (1000,1100):380, (1100,1200):410, (1200,1300):450, (1300,1700):450},
        (1.6, 1.7):    {(800,900):370, (900,1000):380, (1000,1100):400, (1100,1200):440, (1200,1300):450, (1300,1700):450},
        (1.7, 1.8):    {(800,900):370, (900,1000):390, (1000,1100):420, (1100,1200):450, (1200,1700):450},
        (1.8, 1.9):    {(800,900):380, (900,1000):410, (1000,1100):450, (1100,1200):450, (1200,1400):450, (1400,1700):450},
        (1.9, float('inf')):{(800,900):390, (900,1000):430, (1000,1100):450, (1100,1200):450, (1200,1400):450, (1400,1700):450}
    }
}

In [54]:
import re

def check_invalid_size_format(df):
    pattern = re.compile(r'^\s*\d+(\.\d+)?\s*x\s*\d+(\.\d+)?\s*x\s*[Cc]\s*$')  # 예: 123x456xC 또는 12.5x45.3xC

    for col in ['Size', 'Size.1']:
        if col in df.columns:
            invalid_values = df[~df[col].astype(str).str.match(pattern)][col].unique()
            if len(invalid_values) > 0:
                print(f"열 '{col}'에서 형식이 다른 값들:")
                print(invalid_values)
            else:
                print(f"열 '{col}'은 모두 올바른 형식입니다.")
        else:
            print(f"열 '{col}'이(가) 데이터프레임에 존재하지 않습니다.")

check_invalid_size_format(train)
check_invalid_size_format(test)

print(train[['Size', 'Size.1']].head())

열 'Size'은 모두 올바른 형식입니다.
열 'Size.1'은 모두 올바른 형식입니다.
열 'Size'은 모두 올바른 형식입니다.
열 'Size.1'은 모두 올바른 형식입니다.
                 Size              Size.1
0   1.49 x 1253.2 x C   1.49 x 1253.2 x C
1     1.178 x 942 x C     1.178 x 924 x C
2  4.018 x 1268.6 x C    4.018 x 1251 x C
3  1.541 x 1221.6 x C  1.541 x 1221.6 x C
4  0.807 x 1545.2 x C    0.807 x 1530 x C


In [55]:
def convert_size_columns(df):
    df = df.copy()
    for col in ['Size', 'Size.1']:
        if col in df.columns:
            # 공백 제거
            cleaned = df[col].astype(str).str.replace(" ", "", regex=False)
            # 정규식으로 숫자 추출
            extracted = cleaned.str.extract(r'^(\d+\.?\d*)x(\d+\.?\d*)x[Cc]$')
            # float 변환 및 곱하기
            area = extracted[0].astype(float) * extracted[1].astype(float)
            df[col] = area
    return df

train = convert_size_columns(train)
test = convert_size_columns(test)

print(train[['Size', 'Size.1']].head())

print("Size 열의 NaN 개수:", train['Size'].isnull().sum())
print("Size.1 열의 NaN 개수:", train['Size.1'].isnull().sum())

        Size     Size.1
0  1867.2680  1867.2680
1  1109.6760  1088.4720
2  5097.2348  5026.5180
3  1882.4856  1882.4856
4  1246.9764  1234.7100
Size 열의 NaN 개수: 0
Size.1 열의 NaN 개수: 0


In [56]:
from sklearn.linear_model import LinearRegression

def fill_length_with_weight(df):
    df = df.copy()

    # 0을 NaN으로 변경
    df['소재길이'] = df['소재길이'].replace(0, np.nan)

    # 회귀 학습용 데이터 준비 (둘 다 값이 있는 경우만)
    valid1 = df[['소재길이', '소재중량']].dropna()
    X1_train = valid1[['소재중량']]
    y1_train = valid1['소재길이']

    # 선형 회귀 모델 학습
    model1 = LinearRegression()
    model1.fit(X1_train, y1_train)

    # 결측치 채우기
    missing = df['소재길이'].isna()
    df.loc[missing, '소재길이'] = model1.predict(df.loc[missing, ['소재중량']])

    return df

train = fill_length_with_weight(train)
print("소재길이 열의 NaN 개수:", train['소재길이'].isnull().sum())

소재길이 열의 NaN 개수: 0


In [59]:
def preprocess(df):
    df = df.copy()
    # 결측치 처리
    most_freq = df['도유'].mode()[0]
    df['도유'].fillna(most_freq, inplace=True)
    df['강종'].fillna('일반', inplace=True)
    for col in ['장력1','스피드1','장력2','스피드2','장력3','스피드3']:
        df[col].fillna(0, inplace=True)

    # 기준 암페어 계산
    def get_amp(r):
        lo_t, hi_t = parse_range(r['두께그룹'])
        lo_w, hi_w = parse_range(r['폭그룹'])
        mat = amp_matrix.get(r['품명그룹'], {})
        for (t_lo, t_hi), wmap in mat.items():
            if lo_t >= t_lo and hi_t <= t_hi:
                for (w_lo, w_hi), amp in wmap.items():
                    if lo_w >= w_lo and hi_w <= w_hi:
                        return amp
        return None

    df['기준_암페어'] = df.apply(get_amp, axis=1)
    df['기준_암페어'].fillna(0, inplace=True)
    df['가공불가_플래그'] = (df['기준_암페어'] == 0).astype(int)

    # 불필요 컬럼 제거
    drop_cols = ['OK','NG','라인','작업장','소유','유형','수량','제품구분','수요가','월','일자','No.','제품번호', '폭그룹', '두께그룹', '마진그룹', '생산지시번호','소재번호']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

    # 범주형 인코딩
    # for c in df.select_dtypes(include=['object']).columns:
    #     df[c] = df[c].astype('category')
    # cat_cols = df.select_dtypes(include=['category']).columns.tolist()
    # oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    # df[cat_cols] = oe.fit_transform(df[cat_cols])
    # df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    return df

In [60]:
# 전처리 적용
X_train = preprocess(train)
X_test  = preprocess(test)

# train/test 컬럼 정렬
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
y = train['OK']

<ipython-input-59-9ead9769ed44>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['도유'].fillna(most_freq, inplace=True)
<ipython-input-59-9ead9769ed44>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [71]:
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])
print(X_test.isnull().sum()[X_test.isnull().sum() > 0])


categorical_features = ['강종','품명그룹', '품명', '재질', '도유']
X_train['강종'] = X_train['강종'].astype('category')
X_train['품명그룹'] = X_train['품명그룹'].astype('category')
X_train['품명'] = X_train['품명'].astype('category')
X_train['재질'] = X_train['재질'].astype('category')
X_train['도유'] = X_train['도유'].astype('category')

X_test['강종'] = X_test['강종'].astype('category')
X_test['품명그룹'] = X_test['품명그룹'].astype('category')
X_test['품명'] = X_test['품명'].astype('category')
X_test['재질'] = X_test['재질'].astype('category')
X_test['도유'] = X_test['도유'].astype('category')


Series([], dtype: int64)
Series([], dtype: int64)


In [70]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from lightgbm import LGBMClassifier
import numpy as np

def evaluate_lgbm(X, y, cv=5):
    model = LGBMClassifier()
    f1_scores = []
    roc_scores = []

    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train, categorical_feature=['강종','품명그룹', '품명', '재질', '도유'])
        y_pred = model.predict(X_val)
        y_prob = model.predict_proba(X_val)[:, 1]

        f1 = f1_score(y_val, y_pred)
        roc = roc_auc_score(y_val, y_prob)

        f1_scores.append(f1)
        roc_scores.append(roc)

    f1_mean = np.mean(f1_scores)
    roc_mean = np.mean(roc_scores)

    weighted_score = (0.4 * f1_mean + 0.6 * roc_mean) ** 2
    print(f'LightGBM: {weighted_score:.5f}')
    print(f'F1 Score: {f1_mean:.5f}')
    print(f'ROC AUC Score: {roc_mean:.5f}')

evaluate_lgbm(X_train, y)

[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4644
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391984 -> initscore=-0.438981
[LightGBM] [Info] Start training from score -0.438981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000725 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4684
[LightGBM] [Info] Number of data p

In [72]:
# 모델 학습 및 예측
model = LGBMClassifier(random_state=42)
model.fit(X_train, y)

y_pred_test = model.predict(X_test)

#파일 다운로드
X_train.to_csv('X_train.csv', index=False)
from google.colab import files
files.download('X_train.csv')

[LightGBM] [Info] Number of positive: 721, number of negative: 1119
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4872
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391848 -> initscore=-0.439552
[LightGBM] [Info] Start training from score -0.439552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def apply_post_rules(X_test, y_pred_test):
    y_pred_test = y_pred_test.copy()

    # 조건 정의
    condition = (
        (X_test['장력1'] == 0) | (X_test['장력1'].isnull()) |
        (X_test['스피드1'] == 0) | (X_test['스피드1'].isnull()) |
        (X_test['장력2'] == 0) | (X_test['장력2'].isnull()) |
        (X_test['스피드2'] == 0) | (X_test['스피드2'].isnull()) |
        (X_test['장력3'] == 0) | (X_test['장력3'].isnull()) |
        (X_test['스피드3'] == 0) | (X_test['스피드3'].isnull())
    )

    if '장력' in X_test.columns:
        condition |= (X_test['장력'] == 0)
    if '스피드' in X_test.columns:
        condition |= (X_test['스피드'] == 0)
    if '품명_FH' in X_test.columns:
        condition |= (X_test['품명_FH'] == 1)

    # 조건 만족하는 경우 무조건 불량품 (OK = 0) 처리
    y_pred_test[condition] = 0

    return y_pred_test


In [73]:
# submission = pd.DataFrame({
#     'id': range(len(y_pred_test)),
#     'OK': (y_pred_test == 1).astype(int),
#     'NG': (y_pred_test == 0).astype(int)
# })
# submission.to_csv('submission.csv', index=False)


# # Colab에서 다운로드
# from google.colab import files
# files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
# from google.colab import files
# import pandas as pd

# # 파일 업로드
# uploaded = files.upload()

# # 업로드된 파일 이름 추출
# file_names = list(uploaded.keys())
# if len(file_names) != 2:
#     raise ValueError("CSV 파일 2개를 업로드해주세요.")

# # 파일 불러오기
# df1 = pd.read_csv(file_names[0])
# df2 = pd.read_csv(file_names[1])

# # id 기준 정렬 (순서 차이 방지)
# df1 = df1.sort_values('id').reset_index(drop=True)
# df2 = df2.sort_values('id').reset_index(drop=True)

# # OK 값이 다른 행 찾기
# diff_mask = df1['OK'] != df2['OK']
# diff_ids = df1.loc[diff_mask, 'id']

# # 결과 출력
# print("OK 값이 다른 샘플 ID 목록:")
# print(diff_ids.tolist())
# print(f"총 {len(diff_ids)}개의 샘플에서 OK 값이 다릅니다.")

Saving subb.csv to subb.csv
Saving submission (1).csv to submission (1) (1).csv
OK 값이 다른 샘플 ID 목록:
[]
총 0개의 샘플에서 OK 값이 다릅니다.
